In [ ]:
import pandas as pd
import os
import json
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [ ]:
DATA_DIR = DATA_DIR = '/content/gdrive/My Drive/Kaggle/Cassava/'
TEST_DIR = DATA_DIR + 'test_images/'
BATCH_SIZE = 1
NUM_CLASSES = 5
SEED = 0
EPOCH = 32
INPUT_SIZE = 512

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
os.listdir(DATA_DIR)

['sample_submission.csv',
 'train.csv',
 'label_num_to_disease_map.json',
 'train_images',
 'test_images',
 'model1_866.h5']

In [ ]:
with open(DATA_DIR + '/label_num_to_disease_map.json', 'r') as file:
    labels = json.load(file)
    
labels

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [ ]:
from keras.models import load_model

model1 = load_model(DATA_DIR + '/model1_866.h5')

In [ ]:
#subm = pd.read_csv(DATA_DIR + 'sample_submission.csv')
subm = pd.read_csv('sample_submission.csv')

In [ ]:
subm.head()

,image_id,label
0,2216849948.jpg,4
1,1000015157.jpg,0


In [ ]:
subm.label = subm.label.astype("str")

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.3,
    fill_mode = 'nearest',
    horizontal_flip = True,
    vertical_flip = True,
    rescale = 1./255,)

test_generator = test_datagen.flow_from_dataframe(
        dataframe = subm,
        directory = TEST_DIR,
        x_col = "image_id" ,
        y_col = "label",
        target_size = (INPUT_SIZE, INPUT_SIZE),
        color_mode = "rgb",
        shuffle = False,
        class_mode = 'categorical',
        batch_size = 1)

filenames = test_generator.filenames
nb_samples = len(filenames)

Found 2 validated image filenames belonging to 2 classes.


In [ ]:
def tta_prediction(model, testX) : 
    tta_steps = 10
    predictions = []
    for i in tqdm(range(tta_steps)) : 
        preds = model.predict_generator(test_generator)
        predictions.append(preds)
    predictions = np.mean(predictions, axis = 0)
    return predictions

In [ ]:
predict = tta_prediction(model1, test_generator)

for i in range(len(predict)) : 
    subm.loc[i].label = np.argmax(predict[i])

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [ ]:
subm.to_csv('submission.csv', index = None)

In [ ]:
result = pd.read_csv('./submission.csv')
result

,image_id,label
0,2216849948.jpg,4
1,1000015157.jpg,0
